In [1]:
'''https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py
Sequence to sequence example in Keras (character-level).
# Data download
English to French sentence pairs.
http://www.manythings.org/anki/fra-eng.zip
'''
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [2]:
# 參數設定
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

In [5]:
# 設定訓練資料檔路徑
#data_path = 'fra-eng/fra.txt'
data_path = 'cmn-eng/cmn.txt'

In [6]:
# 讀取資料檔，並將所有單字整理為字典，分別為英文及中文字典，注意，英文為字母的集合，非單字(Word)
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path, encoding ='utf8').read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [7]:
# 字典排序            
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

In [8]:
# 計算編碼器、解碼器的最大長度
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [9]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 73
Number of unique output tokens: 2640
Max sequence length for inputs: 31
Max sequence length for outputs: 22


In [10]:
# 以dict儲存字典單字及序號
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [11]:
# 設定編碼器、解碼器input起始值(均為0矩陣)
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [12]:
# 設定 encoder_input、decoder_input對應的順序    
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [13]:
# 建立 encoder LSTM 隱藏層
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [14]:
# 捨棄 output，只保留記憶狀態 h 及 c
encoder_states = [state_h, state_c]

In [15]:
# 建立 decoder LSTM 隱藏層
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# decoder 記憶狀態不會在訓練過程使用，只會在推論(Inference)使用
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [16]:
# 定義模型，由 encoder_input_data 及 decoder_input_data 轉換為 decoder_target_data 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 訓練
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 8000 samples, validate on 2000 samples
Epoch 1/100





8000/8000 [==============================] - 91s 11ms/step - loss: 2.0531 - val_loss: 2.5451
Epoch 2/100
8000/8000 [==============================] - 83s 10ms/step - loss: 1.9228 - val_loss: 2.4361
Epoch 3/100
8000/8000 [==============================] - 84s 11ms/step - loss: 1.8093 - val_loss: 2.3559
Epoch 4/100
8000/8000 [==============================] - 84s 11ms/step - loss: 1.7416 - val_loss: 2.2714
Epoch 5/100
8000/8000 [==============================] - 86s 11ms/step - loss: 1.6617 - val_loss: 2.1826
Epoch 6/100
8000/8000 [==============================] - 85s 11ms/step - loss: 1.5779 - val_loss: 2.1383
Epoch 7/100
8000/8000 [==============================] - 82s 10ms/step - loss: 1.5097 - val_loss: 2.0689
Epoch 8/100
8000/8000 [==============================] - 84s 11ms/step - loss: 1.4491 - val_loss: 2.0144
Epoch 9/

8000/8000 [==============================] - 82s 10ms/step - loss: 0.4051 - val_loss: 2.0008
Epoch 56/100
8000/8000 [==============================] - 81s 10ms/step - loss: 0.3954 - val_loss: 1.9999
Epoch 57/100
8000/8000 [==============================] - 82s 10ms/step - loss: 0.3857 - val_loss: 2.0233
Epoch 58/100
8000/8000 [==============================] - 81s 10ms/step - loss: 0.3768 - val_loss: 2.0272
Epoch 59/100
8000/8000 [==============================] - 81s 10ms/step - loss: 0.3676 - val_loss: 2.0305
Epoch 60/100
8000/8000 [==============================] - 81s 10ms/step - loss: 0.3583 - val_loss: 2.0411
Epoch 61/100
8000/8000 [==============================] - 81s 10ms/step - loss: 0.3510 - val_loss: 2.0462
Epoch 62/100
8000/8000 [==============================] - 83s 10ms/step - loss: 0.3419 - val_loss: 2.0506
Epoch 63/100
8000/8000 [==============================] - 81s 10ms/step - loss: 0.3347 - val_loss: 2.0711
Epoch 64/100
8000/8000 [==============================] - 8

In [17]:
# 儲存模型及結果
model.save('s2s.h5')

/Applications/anaconda3/lib/python3.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [18]:
# 推論(Inference)
# 過程如下:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

In [19]:
# 定義編碼器取樣模型
encoder_model = Model(encoder_inputs, encoder_states)

In [22]:
# 定義解碼器的input
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [23]:
# 定義解碼器 LSTM 模型
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

In [24]:
# 以編碼器的記憶狀態 h 及 c 為解碼器的記憶狀態  
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [25]:
# 建立反向的 dict，才能透過查詢將數值轉回文字
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [26]:
# 模型預測，並取得翻譯結果(中文)    
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [27]:
# 測試100次
for seq_index in range(100):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('*')
    print('Input sentence:', input_texts[seq_index])
    try:
        print('Decoded sentence:', decoded_sentence)
    except:
        # 出現亂碼，以?取代
        print('Decoded sentence:', decoded_sentence.encode('ascii', 'replace'))
        #print("error:", sys.exc_info()[0])

*
Input sentence: Hi.
Decoded sentence: 你好。

*
Input sentence: Hi.
Decoded sentence: 你好。

*
Input sentence: Run.
Decoded sentence: 你用跑的。

*
Input sentence: Wait!
Decoded sentence: 等等！

*
Input sentence: Hello!
Decoded sentence: 你好。

*
Input sentence: I try.
Decoded sentence: 让我来。

*
Input sentence: I won!
Decoded sentence: 我會了。

*
Input sentence: Oh no!
Decoded sentence: 不会吧。

*
Input sentence: Cheers!
Decoded sentence: 乾杯!

*
Input sentence: He ran.
Decoded sentence: 他跑了。

*
Input sentence: Hop in.
Decoded sentence: 抱抱汤姆。

*
Input sentence: I lost.
Decoded sentence: 我迷失了。

*
Input sentence: I quit.
Decoded sentence: 我退出。

*
Input sentence: I'm OK.
Decoded sentence: 我沒事。

*
Input sentence: Listen.
Decoded sentence: 听着。

*
Input sentence: No way!
Decoded sentence: 不可能！

*
Input sentence: No way!
Decoded sentence: 不可能！

*
Input sentence: Really?
Decoded sentence: 你确定？

*
Input sentence: Try it.
Decoded sentence: 试试吧。

*
Input sentence: We try.
Decoded sentence: 我们来试试。

*
Input sentence: 